In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

In [ ]:
image = cv2.imread('base_image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image,(600, 200))

In [ ]:
img_to_tensor = tf.convert_to_tensor(image, dtype=tf.float32)

In [ ]:
processed_image = tf.keras.applications.vgg16.preprocess_input(img_to_tensor)

In [ ]:
pre_trained_model = vgg19.VGG19(weights="imagenet", include_top=False)

In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)

In [ ]:
content_layer = "block5_conv1"
content_model_output = Model(inputs = pre_trained_model.inputs, outputs = pre_trained_model.get_layer(content_layer).output)

In [ ]:
content_model_output.summary()

In [ ]:
class content_model(tf.keras.Model):       
    def __init__(self,content_model,shape):
        super().__init__()     
        self.pre_trained_model = content_model
        self.white_image = tf.Variable(tf.ones(shape = shape))
        self.gradients = []
    
    def compile(self, optimizer):
        super().compile()
        self.optimizer = optimizer
    
    def contenidopapurri(self,image):    
        base_pred = self.pre_trained_model(image)     
        white_pred = self.pre_trained_model(self.white_image)         
        return tf.reduce_sum(tf.square(white_pred - base_pred))
        
    def train_step(self, data):            
        with tf.GradientTape() as tape:
            tape.watch(self.white_image)  
            content_loss = self.contenidopapurri(data)     
        grads = tape.gradient(content_loss, [self.white_image])[0]       
        self.optimizer.apply_gradients([(grads, self.white_image)])   
        self.gradients.append(grads)
        return {"loss": content_loss}

In [ ]:
model = content_model(content_model_output,shape = (1,200,600,3))

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD())

In [ ]:
model.fit(x = np.expand_dims(processed_image,axis = 0),epochs = 50)